In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
from random import randint


# start your browser

browser = webdriver.Chrome('./chromedriver')


browser.implicitly_wait(15)
wait = WebDriverWait(browser, 30)

# enter the URL you would like to scrape

urls = [
        #'file:///Users/chenchiungyi/Documents/Koopwoningen%20Amsterdam%20-%20Huizen%20te%20koop%20in%20Amsterdam%20[funda].htm',
    
        'http://www.funda.nl/koop/amsterdam/p'
    ]


data1 = []
data2 = []


# enter the pagerange you would like to scrape

pagerange = range(1, 2)

# start scraping

for url in urls:
    for page in pagerange:
        browser.get(url + str(page))
        # wait for the page to load
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.search-result-content")))

        for item in browser.find_elements_by_css_selector("div.search-result-content"):


            try:
                zipcode1, zipcode2, city = item.find_element_by_css_selector("h4.search-result__header-subtitle").text.split(" ", 2)
                zipcode = zipcode1 + " " + zipcode2

                street_zipcode_city = city
                #item.find_element_by_css_selector("h3.search-result-title").text

                price = item.find_element_by_css_selector("span.search-result-price").text.lstrip('€ ').rstrip(
                    ' k.k,').replace('.', '')

                surface, rooms = item.find_element_by_css_selector("ul.search-result-kenmerken").text.replace('\n', '').replace('m²', '').split(" ", 1)
                rooms = rooms.replace('kamer', '').replace('s', '')

                link = item.find_element_by_css_selector("div.search-result__header-title-col>a").get_attribute('href')

                data1.append({
                    "street_zipcode_city": street_zipcode_city,
                    "zipcode": zipcode,
                    "city": city,
                    "price": price,
                    "surface": surface,
                    "rooms": rooms,
                    "link": link,
                })

            except ValueError:
                pass

        time.sleep(randint(10, 15))

for j in range(len(data1)-7):
    
    browser.get(data1[j]['link'])
    all_data = []
    for item in browser.find_elements_by_css_selector("dl.object-kenmerken-list"):


        try:
            temp_text = item.text.split()
            all_data.append(temp_text)
    
        except ValueError:
            pass
    
    
    
    if 'VvE' in all_data[0]:
        vve = all_data[0].index('VvE')
        vv_in = all_data[0][vve+1]
    else:
        vve_in = 0
    
    const_index = all_data[1].index('Bouwjaar')
    inh_index = all_data[2].index('Inhoud')
    
    if 'Aantal' in all_data[3]:
        aan_in = all_data[3].index('Aantal')
    
    if 'Gelgen' in all_data[3]:
        gel = all_data[3].index('Gelegen')
        gel_in = all_data[3][gel+2]
    else:
        gel_in = 'na'
    
    
    
    #organize data
    data2.append({
    "per_m2": all_data[0][10],
    "VvE": vve_in,
    "Type": all_data[1][1],
    "Const_yaer": all_data[1][const_index],
    "Volume": all_data[2][-2],
    "bedrooms": all_data[3][4],
    "located": all_data[3][aan_in+2],
    "floors": gel_in,
    "energy_lab": all_data[4][1]
                    
            })

    time.sleep(randint(10, 15))

        
browser.close()
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
mergedDf = df1.merge(df2, left_index=True, right_index=True)
mergedDf.to_csv("AmsterdamPage"+str(min(pagerange))+"to"+str(max(pagerange))+".csv", sep=';', encoding='utf-8')
print(mergedDf)


AttributeError: 'NoneType' object has no attribute 'is_displayed'

In [12]:
print(data1)

[{'street_zipcode_city': 'Amsterdam', 'zipcode': '1065 AG', 'city': 'Amsterdam', 'price': '500000', 'surface': '146', 'rooms': ' / 102 6 ', 'link': 'https://www.funda.nl/koop/amsterdam/huis-41992114-pieter-borstraat-33/?navigateSource=resultlist'}, {'street_zipcode_city': 'Amsterdam', 'zipcode': '1060 PL', 'city': 'Amsterdam', 'price': '450000', 'surface': '125', 'rooms': ' / 98 5 ', 'link': 'https://www.funda.nl/koop/amsterdam/huis-41992009-lomondlaan-70/?navigateSource=resultlist'}, {'street_zipcode_city': 'Amsterdam', 'zipcode': '1086 VB', 'city': 'Amsterdam', 'price': '695000', 'surface': '148', 'rooms': ' / 149 6 ', 'link': 'https://www.funda.nl/koop/amsterdam/huis-41990728-brigantijnkade-40/?navigateSource=resultlist'}, {'street_zipcode_city': 'Amsterdam', 'zipcode': '1024 VL', 'city': 'Amsterdam', 'price': '535000', 'surface': '133', 'rooms': ' / 113 5 ', 'link': 'https://www.funda.nl/koop/amsterdam/huis-41990547-duinlustpad-7/?navigateSource=resultlist'}, {'street_zipcode_city'